In [1]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.pipeline import Pipeline
import time
import nltk
from nltk import tokenize as tok

In [2]:
import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [3]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [4]:
# This step was performed in the sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only1(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in tok.sent_tokenize(text) for word in tok.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [5]:
# This step was performed in the sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")

In [6]:


# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']
readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

[' where', ' have', '#savetheplanet', " might've", " wouldn't", ' own', 'yourselves', 'them', 'had', "they've", 'hers', ' for', "doesn't", 'him', "we're", 'no', " couldn't", 'up', " don't", 'because', 'some', 'm', " we'd", 'http', 'such', 'don', 'hasn', "here's", ' how', " they're", "they're", " we'll", 'ma', "we'd", " what's", 'of', ' since', 'do', 'doesn', ' about', " why's", " it's", ' wants', 'how', ' me', 'and', ' however', ' so', 'this', ' tis', ' if', '#bushfiresAustralia', 'has', " we're", 'if', "she'll", 've', 'in', ' but', ' other', 'out', " she'd", 'ourselves', ' that', "he'll", 'mightn', 'shouldn', ' its', 'its', 'to', ' been', 'after', ' only', ' because', ' what', 'an', 'all', '#FridaysForFuture', " doesn't", "she's", 'or', ' no', 't', 'below', 'any', 'you', ' and', 'just', 'the', "you're", ' able', ' cannot', ' with', "needn't", '#globalwarming', 'while', "couldn't", 'why', ' across', " why'd", ' say', 'he', ' am', ' to', ' this', '#ActOnClimate', " how'd", ' after', "sh

In [7]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [8]:
# # Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y.shape
encoded_y

array([ 5,  5,  5, ..., 10, 10, 10])

In [9]:

tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
#X_Vect = tf_vectorizer.fit_transform(X)



In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.30)#, random_state=42, stratify=y)

In [11]:
X_train.shape

(50683,)

In [12]:
y_train.shape

(50683,)

In [13]:
X_test.shape

(21722,)

In [14]:
y_test.shape

(21722,)

In [15]:
text_clf_pipe = Pipeline([
    ('tfidf',TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only)),
    ('clf',MNB(alpha=0.01))])

In [16]:
%time text_clf_pipe.fit(X_train, y_train)  

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'across', 'actonclimate', 'ai', 'almost', 'also', 'among', 'bushfires', 'bushfiresaustralia', 'ca', 'climateaction', 'climatechange', 'climatecrisis', 'climatestrike', 'dear', 'either', 'else', 'environment', 'ever', 'every', 'fridaysforfuture', 'get', 'globalwarming', 'got', 'greennewdeal', 'however', 'least', 'let', 'like', 'likely', 'may', 'might', 'must', "n't", 'need', 'neither', 'often', 'rather', 'said', 'savetheplanet', 'say', 'says', 'sha', 'since', 'sustainability', 'tis', 'twas', 'us', 'wants', 'wo', 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Wall time: 19.4 s


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.9, max_features=None,
                                 min_df=0.0, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=[' where', ' have',
                                             '#savetheplanet', " might've",
                                             " wouldn't", '...
                                             'no', " couldn't", 'up', " don't",
                                             'because', 'some', 'm', " we'd",
                                             'http', 'such', 'don', 'hasn',
                                      

In [17]:
predictions = text_clf_pipe.predict(X_test)

In [18]:
print(confusion_matrix(y_test,predictions))

[[ 273    0   21  105   29   74    7    5    6   34    2   10]
 [   0  112    1   99   16    9    3    1    1    0    0    0]
 [  26    1  711  446  102  467   30   41   16   28    9   63]
 [  34   46  292 3127  206  508  136   49  152  108   26  121]
 [  25   18   88  533  494  208   25   17   36   54    5   13]
 [   9    9  136  314   99 4700   14  169   19   49   12   12]
 [   3    3   50  368   27   62  720    4   26   17   27  138]
 [   2    3   42   55   15  504   16  242    6    5    7    7]
 [   1    4   13  174   30   57   32    7  867    7    6   13]
 [  13    0   25  213   49  122   10    9   12  937    5    8]
 [   1    0   12   81    1   32   61    6   23    1  182   30]
 [   0    0   61  269   14   30  132   12   28   13   22 1137]]


In [19]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.71      0.48      0.57       566
           1       0.57      0.46      0.51       242
           2       0.49      0.37      0.42      1940
           3       0.54      0.65      0.59      4805
           4       0.46      0.33      0.38      1516
           5       0.69      0.85      0.76      5542
           6       0.61      0.50      0.55      1445
           7       0.43      0.27      0.33       904
           8       0.73      0.72      0.72      1211
           9       0.75      0.67      0.71      1403
          10       0.60      0.42      0.50       430
          11       0.73      0.66      0.70      1718

    accuracy                           0.62     21722
   macro avg       0.61      0.53      0.56     21722
weighted avg       0.61      0.62      0.61     21722



              precision    recall  f1-score   support

           0       0.71      0.51      0.60       549
           1       0.63      0.51      0.57       278
           2       0.51      0.36      0.42      1972
           3       0.52      0.66      0.58      4810
           4       0.47      0.34      0.39      1514
           5       0.70      0.85      0.77      5562
           6       0.57      0.50      0.53      1373
           7       0.43      0.28      0.34       901
           8       0.73      0.67      0.70      1262
           9       0.76      0.64      0.69      1380
          10       0.61      0.39      0.47       407
          11       0.77      0.66      0.71      1714

    accuracy                           0.62     21722
    macro avg      0.62      0.53      0.56     21722
    weighted avg   0.62      0.62      0.61     21722


In [20]:
metrics.accuracy_score(y_test,predictions)

0.6215818064634933

- Accuracy is -- 0.4743117576650401 without Tokenization and Stopwords
- Accuracy is -- 0.4820458521314796 with Tokenization and Stopwords

In [21]:
print('MultinomialNB Train: %.3f' % text_clf_pipe.score(X_train, y_train))
print('MultinomialNB Test Acc: %.3f' % text_clf_pipe.score(X_test, y_test))

MultinomialNB Train: 0.845
MultinomialNB Test Acc: 0.622


In [22]:
strVal = ["Cuyahoga county eliminates use of plastic bags","Weather is still warm in winters and is not freezing"]
# Value = tf_vectorizer.transform(strVal)
# Value.shape

In [23]:
#Use this way to obtain the value from encoding
inv_s = label_encoder.inverse_transform(text_clf_pipe.predict(strVal).astype(int).ravel())
inv_s

array(['#environment', '#globalwarming'], dtype=object)

In [24]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'tfidf__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf_pipe, parameters_svm, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(gs_clf.best_score_))
print("Best parameters: ", gs_clf.best_params_)
# param_grid = {'alpha': [0.001, 0.01, 0.1, 1]}

# grid = GridSearchCV(MNB(), param_grid, cv=10)
# grid.fit(X_train, y_train)
# print("Best cross-validation score: {:.2f}".format(grid.best_score_))
# print("Best parameters: ", grid.best_params_)

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'t", "'ve", 'able', 'across', 'actonclimate', 'ai', 'almost', 'also', 'among', 'bushfires', 'bushfiresaustralia', 'ca', 'climateaction', 'climatechange', 'climatecrisis', 'climatestrike', 'dear', 'either', 'else', 'environment', 'ever', 'every', 'fridaysforfuture', 'get', 'globalwarming', 'got', 'greennewdeal', 'however', 'least', 'let', 'like', 'likely', 'may', 'might', 'must', "n't", 'need', 'neither', 'often', 'rather', 'said', 'savetheplanet', 'say', 'says', 'sha', 'since', 'sustainability', 'tis', 'twas

Best cross-validation score: 0.67
Best parameters:  {'clf__alpha': 0.01, 'tfidf__ngram_range': (1, 2), 'tfidf__use_idf': False}


Best cross-validation score: 0.67
Best parameters:  {'clf__alpha': 0.01, 'tfidf__ngram_range': (1, 2), 'tfidf__use_idf': False}

In [25]:
gs_clf.predict(strVal)

array([6, 3])

In [26]:
#Use this way to obtain the value from encoding
inv_s = label_encoder.inverse_transform(gs_clf.predict(strVal).astype(int).ravel())
inv_s

array(['#environment', '#climatechange'], dtype=object)

In [37]:
import nltk
# nltk.download()

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('mnb', MultinomialNB(fit_prior=False))])

text_mnb_stemmed = text_mnb_stemmed.fit(X_train, y_train)

predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test)

np.mean(predicted_mnb_stemmed == y_test)

0.5764202191326766

In [32]:
print(classification_report(y_test,predicted_mnb_stemmed))

              precision    recall  f1-score   support

           0       0.88      0.16      0.28       566
           1       0.78      0.20      0.32       242
           2       0.63      0.21      0.31      1940
           3       0.43      0.79      0.55      4805
           4       0.70      0.15      0.24      1516
           5       0.62      0.89      0.73      5542
           6       0.70      0.39      0.50      1445
           7       0.65      0.17      0.27       904
           8       0.81      0.44      0.57      1211
           9       0.87      0.46      0.61      1403
          10       0.88      0.27      0.41       430
          11       0.83      0.60      0.70      1718

    accuracy                           0.58     21722
   macro avg       0.73      0.39      0.46     21722
weighted avg       0.65      0.58      0.54     21722



In [33]:
metrics.accuracy_score(y_test,predicted_mnb_stemmed)

0.5764202191326766

In [35]:
inv_s = label_encoder.inverse_transform(text_mnb_stemmed.predict(strVal).astype(int).ravel())
inv_s

array(['#environment', '#globalwarming'], dtype=object)

In [40]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'mnb__alpha': (1e-2, 1e-3)}
gs_clf1 = GridSearchCV(text_mnb_stemmed, parameters_svm, n_jobs=-1)
gs_clf1 = gs_clf1.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(gs_clf1.best_score_))
print("Best parameters: ", gs_clf1.best_params_)

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best cross-validation score: 0.69
Best parameters:  {'mnb__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
